#### Структури для роботи з великими обсягами даних в Python

In [ ]:
import pandas as pd
import numpy as np
import missingno as msno
from tabulate import tabulate
from datetime import time as dt_time
import time as tm

print("Setup complete")

file = "household_power_consumption.csv"

In [ ]:
def print_head(data, n=5):
    if isinstance(data, pd.DataFrame):
        table_data = data.head(n).values.tolist()
        headers = data.columns.tolist()
        print(f"Перші ${n} рядків DataFrame:")
        print(tabulate(table_data, headers=headers, tablefmt='pretty'))
    elif isinstance(data, np.ndarray):
        table_data = data[:n].tolist()
        print(f"\nПерші ${n} рядків NumPy масиву:")
        print(tabulate(table_data, tablefmt='pretty'))
    else:
        print("Непідтримуваний тип даних. Підтримуються лише DataFrame або NumPy масив.")

def create_numpy_arr(file1):
    data = np.genfromtxt(file1, delimiter=',')
    data[:,2:8] = data[:,2:8].astype(np.float64)
    return data

def create_data_frame(file1):
    dtype = {
        'Global_active_power': float,
        'Global_reactive_power': float,
        'Voltage': float,
        'Global_intensity': float,
        'Sub_metering_1': float,
        'Sub_metering_2': float,
        'Sub_metering_3': float
    }

    na_values = ['?']
    df1 = pd.read_csv(file1, dtype=dtype, na_values=na_values)

    df1['Date'] = pd.to_datetime(df1['Date'], dayfirst=True)
    df1['Time'] = pd.to_datetime(df1['Time'], format='%H:%M:%S').dt.time
    df1 = df1.dropna().reset_index(drop=True)
    #msno.matrix(df1)

    return df1

df = create_data_frame(file)
np_arr = df.to_numpy()
print_head(create_numpy_arr(file),10) # dirty np
print_head(df,10) # clean df
print_head(np_arr,10) # clean np

1. Обрати всі домогосподарства, у яких загальна активна споживана потужність перевищує 5 кВт.

In [ ]:
def filter_by_power(df1):
    filtered_df1 = df1[df1['Global_active_power'] > 5.0]
    return filtered_df1

def filter_by_power_np(np_arr1):
    filtered_df1 = np_arr1[np_arr1[:, 2] > 5.0]
    return filtered_df1

print("\nЗадача 1")
start_time = tm.time()
filtered_df = filter_by_power(df)
print_head(filtered_df)
end_time = tm.time() - start_time
print(f"Час виконання:\nPd: {end_time} сек")

start_time_np = tm.time()
filtered_df_np = filter_by_power_np(np_arr)
print_head(filtered_df_np)
end_time_np = tm.time() - start_time_np
print(f"Час виконання:\nNp:{end_time_np} сек")

##### 2. Обрати всі домогосподарства, у яких вольтаж перевищую 235 В.

In [ ]:
# Задача 2
def filter_by_voltage(df1):
    filtered_df1 = df1[df1['Voltage'] > 235]
    return filtered_df1

# Задача 2
def filter_by_voltage_np(np_arr1):
    filtered_df1 = np_arr1[np_arr1[:, 4] > 235]
    return filtered_df1


print("\nЗадача 2")
start_time = tm.time()
filtered_df = filter_by_voltage(df)
print_head(filtered_df)
end_time = tm.time() - start_time
print(f"Час виконання:\nPd: {end_time} сек")

start_time_np = tm.time()
filtered_df_np = filter_by_voltage_np(np_arr)
print_head(filtered_df_np)
end_time_np = tm.time() - start_time_np
print(f"Час виконання:\nNp:{end_time_np} сек")


##### 3. Обрати всі домогосподарства, у яких сила струму лежить в межах 19-20 А, для них виявити ті, у яких пральна машина та холодильних споживають більше, ніж бойлер та кондиціонер.

In [ ]:
def filter_by_current_and_consumption(df1):
    filtered_df1 = df1[(df1['Global_intensity'] >= 19) & (df1['Global_intensity'] <= 20)]
    filtered_df1 = filtered_df1[(filtered_df1['Sub_metering_2'] > filtered_df1['Sub_metering_3'])] 
    return filtered_df1

def filter_by_current_and_consumption_np(np_arr1):
    filtered_df1 = np_arr1[(np_arr1[:, 5] >= 19) & (np_arr1[:, 5] <= 20)]
    filtered_df1 = filtered_df1[(filtered_df1[:, 7] > filtered_df1[:, 8])]
    return filtered_df1

print("\nЗадача 3")
start_time = tm.time()
filtered_df = filter_by_current_and_consumption(df)
print_head(filtered_df)
end_time = tm.time() - start_time
print(f"Час виконання:\nPd: {end_time} сек")

start_time_np = tm.time()
filtered_df_np = filter_by_current_and_consumption_np(np_arr)
print_head(filtered_df_np)
end_time_np = tm.time() - start_time_np
print(f"Час виконання:\nNp:{end_time_np} сек")


##### 4. Обрати випадковим чином 500000 домогосподарств (без повторів елементів вибірки), для них обчислити середні величини усіх 3-х груп споживання електричної енергії, а також

In [ ]:
# Задача 4
def random_sample_average(df1, n=500000):
    sampled_indices = np.random.choice(df1.index, n, replace=True)
    sampled_df = df1.loc[sampled_indices]
    averages1 = sampled_df[['Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']].mean()
    return averages1

# Задача 4
def random_sample_average_np(np_arr1, n=500000):
    sampled_np = np_arr1[np.random.choice(np_arr1.shape[0], n, replace=True), 6:9]
    averages1 = np.mean(sampled_np, axis=0)
    return averages1

print("\nЗадача 4")
start_time = tm.time()
averages = random_sample_average(df, n=500000)
print(averages)
end_time = tm.time() - start_time
print(f"Час виконання:\nPd: {end_time} сек")

start_time_np = tm.time()
averages = random_sample_average_np(np_arr, n=500000)
print(averages)
end_time_np = tm.time() - start_time_np
print(f"Час виконання:\nNp:{end_time_np} сек")


##### 5. Обрати ті домогосподарства, які після 18-00 споживають понад 6 кВт за хвилину в середньому, серед відібраних визначити ті, у яких основне споживання електроенергії у вказаний проміжок часу припадає на пральну машину, сушарку, холодильник та освітлення (група 2 є найбільшою), а потім обрати кожен третій результат із першої половини та кожен четвертий результат із другої половини.

In [ ]:
# Задача 5
def complex_filter(df1):
    filtered_df1 = df1[(df1['Time'] > pd.to_datetime('18:00:00').time()) & (df1['Global_active_power'] > 6)
                       & (df1['Sub_metering_2'].gt(df1['Sub_metering_1'], df1['Sub_metering_3']))]

    fh = filtered_df1.iloc[:len(filtered_df1) // 2]
    sh = filtered_df1.iloc[len(filtered_df1) // 2:]
    result = pd.concat([fh.iloc[2::3], sh.iloc[3::4]])

    return result

def complex_filter_np(np_arr1):
        # Імітуймо умовний фільтр для numpy array
    condition = (np_arr1[:, 1].astype('O').astype(dt_time) >= dt_time(18, 0, 0)) & \
                (np_arr1[:, 2] > 6) & \
                (np_arr1[:, 7] > np_arr1[:, 6]) & \
                (np_arr1[:, 7] > np_arr1[:, 8])
    
    filtered_array = np_arr1[condition]

    fh = filtered_array[:len(filtered_array) // 2]
    sh = filtered_array[len(filtered_array) // 2:]
    result = np.concatenate([fh[2::3], sh[3::4]])
    return result

print("\nЗадача 5:")
start_time = tm.time()
filtered_df = complex_filter(df)
print_head(filtered_df)
end_time = tm.time() - start_time
print(f"Час виконання:\nPd: {end_time} сек")

start_time_np = tm.time()
filtered_np = complex_filter_np(np_arr)
print_head(filtered_np)
end_time_np = tm.time() - start_time_np
print(f"Час виконання:\nNp:{end_time_np} сек")
